# LangChain Introduction

**LangChain** is a software development framework designed to simplify the creation of applications that use large language models (LLMs) like OpenAI's GPT. It's written in Python and JavaScript, and it was launched as an open-source project in October 2022 by Harrison Chase, a developer working at the machine learning startup Robust Intelligence.

The framework offers a broad range of capabilities and features for developers, allowing them to harness the power of LLMs for a variety of applications. Here are some key features of LangChain:

**Ease of Use**: LangChain simplifies the process of integrating LLMs into applications. This makes it easier for developers to create and deploy applications that leverage language models for a variety of tasks, including document analysis, text summarization, and chatbot functionality.

**Flexibility**: LangChain includes integrations with a variety of systems and services, including Amazon, Google, and Microsoft Azure cloud storage; API wrappers for news, movie information, and weather; and support for multiple web scraping subsystems and templates.

**Chain Mechanism:** At the heart of LangChain is the "Chain" mechanism, which is a sequence of operations that are performed on a given input. The output of one operation can be used as the input for the next, allowing for complex, multi-step processes to be created and executed easily.

**Memory Functionality:** LangChain supports the concept of a memory for chain objects, allowing data to persist across multiple calls. This makes the chain a stateful object, which can be useful for certain types of applications.

**Customizable Chains:** While LangChain provides many predefined chains, developers also have the flexibility to create custom chains tailored to their specific needs. This can be done by subclassing the Chain class and implementing the required methods.


### Why we use Langchain ?

Langchain is a python library that helps you interact with LLMs like ChatGPT and connect it external data and apps.

At its core, LangChain is a framework built around LLMs. We can use it for chatbots, Generative Question-Answering (GQA), summarization, and much more.

The core idea of the library is that we can “chain” together different components to create more advanced use cases around LLMs. Chains may consist of multiple components from several modules:

- Prompt templates: Prompt templates are templates for different types of prompts. Like “chatbot” style templates, ELI5 question-answering, etc
- LLMs: Large language models like GPT-3, BLOOM, etc
- Agents: Agents use LLMs to decide what actions should be taken. Tools like web search or calculators can be used, and all are packaged into a logical loop of operations.
- Memory: Short-term memory, long-term memory.
We will dive into each of these in much more detail in upcoming chapters of the LangChain handbook.

For now, we’ll start with the basics behind prompt templates and LLMs. We’ll also explore two LLM options available from the library, using models from Hugging Face Hub or OpenAI.

## Generating Text

In [18]:
#!pip install huggingface_hub

In [229]:
from langchain_openai import OpenAI
from langchain import HuggingFaceHub
from langchain_core.runnables import Runnable
from langchain_core.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain, LLMChain, LLMMathChain, TransformChain, SequentialChain
from langchain_core.output_parsers import StrOutputParser
from langchain import FewShotPromptTemplate
from langchain.callbacks import get_openai_callback
import inspect
import re
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [29]:
llm =   OpenAI(model_name="gpt-3.5-turbo-instruct", api_key=OPENAI_API_KEY, temperature=0.7)

Optional approach if you want to use LLM models for free from HuggingFace - You must first install the **huggingface_hub** module and get a HuggingFace hub token. Can be slow!

In [ ]:
# initialize Hub LLM
hub_llm = HuggingFaceHub(
        repo_id='google/flan-t5-xl',
    model_kwargs={'temperature':1e-10}
)

# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

# ask the user question about NFL 2010
print(llm_chain.run(question))

### Prompts and Prompt templates ✏️

Before understanding the concept of prompt templates it is important that you must be aware about what does prompt actually means. Prompt is simply a textual instruction which we give to a model to provide some specific output.

To better understand this let us assume that we want some outline about tennis. So for this our prompt could something be like "Write me an outline on Tennis". But what if we want to again want to get some outline about some other sport let say cricket. In such kind of scenarios the naive approach would be to simply rewrite the prompt with updated sport.

But, if you are an AI Engineer you would be aware about the concept of code reproducibility and in the above mentioned naive approach this concept is getting violated as for every different city we are forced to rewrite the entire prompts with updated sport, so to make the process of creating the prompts efficient we use prompts templates.

*Prompt templates are like ready-made templates which contains contextual information about the input parameter, where input parameter is simply the input provided by the end user. The below mentioned code snipped will help you understand how we can create prompts efficiently.*

## Structure of a Prompt

A prompt can consist of multiple components:

* Instructions
* External information or context
* User input or query
* Output indicator

Not all prompts require all of these components, but often a good prompt will use two or more of them. Let's define what they all are more precisely.

**Instructions** tell the model what to do, typically how it should use inputs and/or external information to produce the output we want.

**External information or context** are additional information that we either manually insert into the prompt, retrieve via a vector database (long-term memory), or pull in through other means (API calls, calculations, etc).

**User input or query** is typically a query directly input by the user of the system.

**Output indicator** is the *beginning* of the generated text. For a model generating Python code we may put `import ` (as most Python scripts begin with a library `import`), or a chatbot may begin with `Chatbot: ` (assuming we format the chatbot script as lines of interchanging text between `User` and `Chatbot`).

Each of these components should usually be placed in the order we've described them. We start with instructions, provide context (if needed), then add the user input, and finally end with the output indicator.

In [48]:
template = PromptTemplate.from_template("Write me an outline on {input_parameter}?")   
user_input = input("Enter sport : ")
prompt = template.format(input_parameter=user_input)
print("Prompt :",prompt)

Enter sport :  tennis


Prompt : Write me an outline on tennis?


In [66]:
prompt = PromptTemplate.from_template("Write me an outline on {input_parameter}?")   
user_input = input("Enter sport : ")
prompt.format(input_parameter=user_input)
# Instantiation using initializer
prompt = PromptTemplate(input_variables=["input_parameter"], template="Write me an outline on {input_parameter}")
print("Prompt :",prompt)

Enter sport :  tennis


Prompt : input_variables=['input_parameter'] template='Write me an outline on {input_parameter}'


We wouldn't typically know what the users prompt is beforehand, so we actually want to add this in. So rather than writing the prompt directly, we create a PromptTemplate with a single input variable query.

## Chains

Now we are going to use a Langchain concept Chains. Chains are responsible for the entire data flow inside Langchain. As we discussed above we are passing dynamic topic input variable to OpenAI. To accommodate this we will be using a chain called LLMChain. There are a bunch of chains supported in Langchain, we will talk about them later

LLMChain takes the prompt from the prompt template we created above and fills it up with the dynamic input before passing to OpenAI LLM. Let's define LLMChain below

In [68]:
chain = LLMChain(llm=llm, prompt=prompt)

Now that we have created a prompt template and a chain we can now input any topic we want. Instead of topic "Tennis" we can input "Cricket" or any other topic of your choice

In [74]:
print(chain.run(input_parameter="Cricket"))



I. Introduction
- Brief history and origin of cricket
- Popularity and global reach of the sport

II. Rules and Objectives of Cricket
- Basic gameplay and scoring system
- Different formats of cricket (Test, One Day, T20)
- Role of the players and their positions on the field

III. Equipment and Playing Field
- Description of the cricket field and its dimensions
- Essential equipment required for playing cricket (bat, ball, stumps, etc.)
- Evolution of cricket equipment over the years

IV. Skills and Techniques
- Batting: grip, stance, and different types of shots
- Bowling: various types of deliveries and techniques
- Fielding: different fielding positions and techniques
- Importance of physical fitness and training in cricket

V. Major Cricket Tournaments and Leagues
- International tournaments such as the ICC Cricket World Cup and T20 World Cup
- Domestic leagues like the Indian Premier League (IPL) and Big Bash League (BBL)
- Impact of these tournaments on the growth and populari

Now let's extend it for a multi-input prompt. Let's generate an introductory paragraph to a blog post with variables title, audience and tone of voice

In [76]:
prompt = PromptTemplate(
    input_variables=["title", "audience", "tone"],
    template="""This program will generate an introductory paragraph to a blog post given a blog title, audience, and tone of voice

    Blog Title: {title}
    Audience: {audience}
    Tone of Voice: {tone}""",
)
chain = LLMChain(llm=llm, prompt=prompt)

In [78]:
print(chain.run(title="Best Activities in Toronto", audience="Millenials", tone="Lighthearted"))



Welcome fellow millenials! Are you ready to discover the best activities and hidden gems that Toronto has to offer? Look no further, because I've compiled a list of must-try experiences that will make your time in this bustling city unforgettable. From rooftop patios to underground speakeasies, get ready to explore Toronto in a whole new light. So put on your adventure hats and let's dive into the best activities in Toronto, all with a lighthearted twist.


## Combining Chains

Often we would want to do multiple tasks using GPT. For example if we wish to generate an outline for a topic and use that outline to write a blog article we need to take the outline created from the first step and copy paste and paste as input to the second step

Instead we can combine chains to achieve this in a single step. We will do this using a different type of chain called Sequential Chain. A sequential chain takes the output from one chain and passes on to the next. We will cover chains in more detail later

In [82]:
prompt = PromptTemplate(
    input_variables=["topic"],
    template="Write me an outline on {topic}",
)

llm = OpenAI(temperature=0.9, max_tokens=-1)

chain_one = LLMChain(llm=llm, prompt=prompt)

second_prompt = PromptTemplate(
    input_variables=["outline"],
    template="""Write a blog article in the format of the given outline 

    Outline:
    {outline}""",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [84]:
overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)
# Run the chain specifying only the input variable for the first chain.
catchphrase = overall_chain.run("Tennis")
print(catchphrase)



> Entering new SimpleSequentialChain chain...


I. Introduction
    A. Explanation of Tennis
    B. Brief history of the sport
    C. Popularity of Tennis worldwide

II. Rules and Equipment
    A. Scoring system
    B. Playing surface and dimensions of the court
    C. Equipment needed (racquets, balls, etc.)
    D. Role of the umpire and linesmen

III. How to Play
    A. Serving
        1. Types of serves (forehand, backhand, overhead)
        2. Proper technique
    B. Groundstrokes
        1. Forehand
        2. Backhand
    C. Volleys
        1. Types of volleys (forehand, backhand)
        2. When to use volleys
    D. Strategy and tactics
        1. Singles vs doubles
        2. Importance of positioning and footwork

IV. Types of Matches
    A. Singles
        1. Scoring and rules
        2. Singles strategies and tactics
    B. Doubles
        1. Scoring and rules
        2. Doubles strategies and tactics
    C. Grand Slam tournaments
        1. Types of Grand Slams
        2

## Revisit the Prompt in Langchain

The prompt template classes in Langchain are built to make constructing prompts with dynamic inputs easier. Of these classes, the simplest is the PromptTemplate. We’ll test this by adding a single dynamic input to our previous prompt, the user query.

In [130]:
template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

With this, we can use the **format** method on our **prompt_template** to see the effect of passing a query to the template.

In [133]:
print(
    prompt_template.format(
        query="Which libraries and model providers offer LLMs?"
    )
)

Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: 


In [135]:
prompt = prompt_template.format(query="Which libraries and model providers offer LLMs?")
chain = prompt_template | llm
chain.invoke(prompt)

"Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library."

### Few Shot Prompt Templates

The success of LLMs comes from their large size and ability to store “knowledge” within the model parameter, which is learned during model training. However, there are more ways to pass knowledge to an LLM. The two primary methods are:

- Parametric knowledge — the knowledge mentioned above is anything that has been learned by the model during training time and is stored within the model weights (or parameters).
- Source knowledge — any knowledge provided to the model at inference time via the input prompt.
Langchain’s FewShotPromptTemplate caters to source knowledge input. The idea is to “train” the model on a few examples — we call this few-shot learning — and these examples are given to the model within the prompt.

Few-shot learning is perfect when our model needs help understanding what we’re asking it to do. We can see this in the following example:

In [142]:
# create our examples
examples = [ #you can have n such example of query-answer pairs
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

If we then pass in the examples and user query, we will get this:

In [144]:
query = "What is the meaning of life?"

print(few_shot_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 



User: How are you?
AI: I can't complain but sometimes I still do.



User: What time is it?
AI: It's time to get a watch.



User: What is the meaning of life?
AI: 


Considering this, we need to balance the number of examples included and our prompt size. Our hard limit is the maximum context size, but we must also consider the cost of processing more tokens through the LLM. Fewer tokens mean a cheaper service and faster completions from the LLM.

The `FewShotPromptTemplate` allows us to vary the number of examples included based on these variables. First, we create a more extensive list of examples:

In [ ]:
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "42"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]

After this, rather than passing the examples directly, we actually use a `LengthBasedExampleSelector` like so:

In [149]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50  # this sets the max length that examples should be
)

We then pass our `example_selector` to the `FewShotPromptTemplate` to create a new — and dynamic — prompt template:

In [153]:
# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

Now if we pass a shorter or longer query, we should see that the number of included examples will vary.

In [156]:
print(dynamic_prompt_template.format(query="How do birds fly?"))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: How do birds fly?
AI: 


Passing a longer question will result in fewer examples being included:

In [159]:
query = """If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?"""

print(dynamic_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?
AI: 


With this, we’re returning fewer examples within the prompt variable. Allowing us to limit excessive token usage and avoid errors from surpassing the maximum context window of the LLM.

An extra utility we will use is this function that will tell us how many tokens we are using in each call. This is a good practice that is increasingly important as we use more complex tools that might make several calls to the API (like agents). It is very important to have a close control of how many tokens we are spending to avoid unsuspected expenditures.

## Let's revisit chains

In [ ]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

Chains are divided in three types: Utility chains, Generic chains and Combine Documents chains. In this edition, we will focus on the first two since the third is too specific (will be covered in due course).

1. Utility Chains: chains that are usually used to extract a specific answer from a llm with a very narrow purpose and are ready to be used out of the box.
2. Generic Chains: chains that are used as building blocks for other chains but cannot be used out of the box on their own.

### Utility Chains

Let's start with a simple utility chain. The `LLMMathChain` gives llms the ability to do math. Let's see how it works!

#### Pro-tip: use `verbose=True` to see what the different steps in the chain are!

In [186]:
lm_math = LLMMathChain.from_llm(llm=llm)

# llm_math = LLMMathChain(llm=llm, verbose=True)

count_tokens(llm_math, "What is 13 raised to the .3432 power?")



> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?```text
13**0.3432
```
...numexpr.evaluate("13**0.3432")...

Answer: 2.4116004626599237
> Finished chain.
Spent a total of 228 tokens


'Answer: 2.4116004626599237'

Let's see what is going on here. The chain recieved a question in natural language and sent it to the llm. The llm returned a Python code which the chain compiled to give us an answer. A few questions arise.. How did the llm know that we wanted it to return Python code? 

**Enter prompts**

The question we send as input to the chain is not the only input that the llm recieves 😉. The input is inserted into a wider context, which gives precise instructions on how to interpret the input we send. This is called a _prompt_. Let's see what this chain's prompt is!

In [191]:
print(llm_math.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



Ok.. let's see what we got here. So, we are literally telling the llm that for complex math problems **it should not try to do math on its own** but rather it should print a Python code that will calculate the math problem instead. Probably, if we just sent the query without any context, the llm would try (and fail) to calculate this on its own. Wait! This is testable.. let's try it out! 🧐

In [195]:
# we set the prompt to only have the question we ask
prompt = PromptTemplate(input_variables=['question'], template='{question}')
llm_chain = LLMChain(prompt=prompt, llm=llm)

# we ask the llm for the answer with no context

count_tokens(llm_chain, "What is 13 raised to the .3432 power?")

Spent a total of 18 tokens


'\n\n6.245297354'

Wrong answer! Herein lies the power of prompting and one of our most important insights so far: 

**Insight**: _by using prompts intelligently, we can force the llm to avoid common pitfalls by explicitly and purposefully programming it to behave in a certain way._

Another interesting point about this chain is that it not only runs an input through the llm but it later compiles Python code. Let's see exactly how this works.

In [203]:
print(inspect.getsource(llm_math._call))

    def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        _run_manager.on_text(inputs[self.input_key])
        llm_output = self.llm_chain.predict(
            question=inputs[self.input_key],
            stop=["```output"],
            callbacks=_run_manager.get_child(),
        )
        return self._process_llm_result(llm_output, _run_manager)



So we can see here that if the llm returns Python code we will compile it with a Python REPL* simulator. We now have the full picture of the chain: either the llm returns an answer (for simple math problems) or it returns Python code which we compile for an exact answer to harder problems. Smart!

Also notice that here we get our first example of **chain composition**, a key concept behind what makes langchain special. We are using the `LLMMathChain` which in turn initializes and uses an `LLMChain` (a 'Generic Chain') when called. We can make any arbitrary number of such compositions, effectively 'chaining' many such chains to achieve highly complex and customizable behaviour.

Utility chains usually follow this same basic structure: there is a prompt for constraining the llm to return a very specific type of response from a given query. We can ask the llm to create SQL queries, API calls and even create Bash commands on the fly 🔥

The list continues to grow as langchain becomes more and more flexible and powerful so we encourage you to [check it out](https://python.langchain.com/v0.2/docs/how_to/) and tinker with the example notebooks that you might find interesting.

*_A Python REPL (Read-Eval-Print Loop) is an interactive shell for executing Python code line by line_

### Generic chains

There are only three Generic Chains in langchain and we will go all in to showcase them all in the same example. Let's go!

Say we have had experience of getting dirty input texts. Specifically, as we know, llms charge us by the number of tokens we use and we are not happy to pay extra when the input has extra characters. Plus its not neat 😉

First, we will build a custom transform function to clean the spacing of our texts. We will then use this function to build a chain where we input our text and we expect a clean text as output.

In [214]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    
    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

Importantly, when we initialize the chain we do not send an llm as an argument. As you can imagine, not having an llm makes this chain's abilities much weaker than the example we saw earlier. However, as we will see next, combining this chain with other chains can give us highly desirable results.

In [217]:
clean_extra_spaces_chain = TransformChain(input_variables=["text"], output_variables=["output_text"], transform=transform_func)

In [219]:
clean_extra_spaces_chain.run('A random text  with   some irregular spacing.\n\n\n     Another one   here as well.')

'A random text with some irregular spacing.\n Another one here as well.'

Great! Now things will get interesting.

Say we want to use our chain to clean an input text and then paraphrase the input in a specific style, say a poet or a policeman. As we now know, the `TransformChain` does not use a llm so the styling will have to be done elsewhere. That's where our `LLMChain` comes in. We know about this chain already and we know that we can do cool things with smart prompting so let's take a chance!

First we will build the prompt template:

In [234]:
template = """Paraphrase this text:

{output_text}

In the style of a {style}.

Paraphrase: """
prompt = PromptTemplate(input_variables=["style", "output_text"], template=template)

And next, initialize our chain:

In [238]:
style_paraphrase_chain = LLMChain(llm=llm, prompt=prompt, output_key='final_output')

Great! Notice that the input text in the template is called 'output_text'. Can you guess why?

We are going to pass the output of the `TransformChain` to the `LLMChain`!

Finally, we need to combine them both to work as one integrated chain. For that we will use `SequentialChain` which is our third generic chain building block.

In [240]:
sequential_chain = SequentialChain(chains=[clean_extra_spaces_chain, style_paraphrase_chain], input_variables=['text', 'style'], output_variables=['final_output'])

Our input is the langchain docs description of what chains are but dirty with some extra spaces all around.

In [243]:
input_text = """
Chains allow us to combine multiple 


components together to create a single, coherent application. 

For example, we can create a chain that takes user input,       format it with a PromptTemplate, 

and then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by 


combining chains with other components.
"""

We are all set. Time to get creative!

In [246]:
count_tokens(sequential_chain, {'text': input_text, 'style': 'a 90s rapper'})

Spent a total of 152 tokens


"Yo, check it - chains be the key, makin' dope apps is easy.\nCombine user input, PromptTemplate, and an LLM, makin' everything crisp and clean.\nStack multiple chains or mix 'em with other components, makin' even crazier machines."

## A note on langchain-hub

`langchain-hub` is a sister library to `langchain`, where all the chains, agents and prompts are serialized for us to use.

In [251]:
from langchain.chains import load_chain

Loading from langchain hub is as easy as finding the chain you want to load in the repository and then using `load_chain` with the corresponding path. We also have `load_prompt` and `initialize_agent`, but more on that later. Let's see how we can do this with our `LLMMathChain` we saw earlier:

In [256]:
llm_math_chain = load_chain('lc://chains/llm-math/chain.json')

What if we want to change some of the configuration parameters? We can simply override it after loading:

In [312]:
llm_math_chain.verbose

In [ ]:
llm_math_chain.verbose = False

In [ ]:
llm_math_chain.verbose